In [42]:
import data
import stack
import transformations as trans
import nibabel as nib
import torch as t
import os
import numpy as np
import open3d as o3d
import pytorch3d as p3

In [43]:
filename = "iFIND00472_iFIND2_201901111210_PHILIPSJ2LF9B9_101_PIH1HR_Survey_32SENSE.s1.nii.gz"
t_image, t_affine, zooms = data.nii_to_torch(filename)

In [44]:
beta = 0.01
first_stack = stack.stack(t_image,t_affine, beta)

In [228]:
I = first_stack.I
X = t.zeros_like(I)
coord = (p_r[0].item(),p_r[1].item(),p_r[2].item())
print(coord)
val = X[int(p_r[0].item()), int(p_r[1].item()),int(p_r[2].item())]

(-303.9285888671875, -303.9285888671875, 2.0)


In [10]:
for i in range(0,I.shape[0]):
    for j in range(0,I.shape[1]):
        p_s = t.tensor([[i],[j],[0],[1]],dtype=t.float32)
        p_r = t.matmul(F_inv[:,:,k],p_s)
        p_s_tilde = t.matmul(F[:,:,k],p_r)
        p_s_diff = (p_s - p_s_tilde)[:3]
        X[int(p_r[0].item()), int(p_r[1].item()),int(p_r[2].item())] += stack.PSN_Gauss(p_s_diff) * I[i,j,k]
        

In [8]:
#p_r_corners = t.zeros(4,1)
p_r_corners = t.zeros(4,8)
n = 0
for k in [0,4]:
    for i in [0,I.shape[0]]:
        for j in [0,I.shape[1]]:
            p_s = t.tensor([[i],[j],[0],[1]],dtype=t.float32)
            corner =  t.matmul(F_inv[:,:,k],p_s)
            p_r_corners[:,n] = corner.transpose(0,1)
            n+=1

In [9]:
print(p_r_corners)

tensor([[ 287.5000,  287.5000, -304.9572, -304.9572,  287.5000,  287.5000,
         -304.9572, -304.9572],
        [ 287.5000, -304.9572,  287.5000, -304.9572,  287.5000, -304.9572,
          287.5000, -304.9572],
        [   2.0000,    2.0000,    2.0000,    2.0000,    2.9091,    2.9091,
            2.9091,    2.9091],
        [   1.0000,    1.0000,    1.0000,    1.0000,    1.0000,    1.0000,
            1.0000,    1.0000]])


In [26]:
x_min, x_max = t.min(p_r_corners[0,:]).item(), t.max(p_r_corners[0,:]).item()
y_min, y_max = t.min(p_r_corners[1,:]).item(), t.max(p_r_corners[1,:]).item()
z_min, z_max = t.min(p_r_corners[2,:]).item(), t.max(p_r_corners[2,:]).item()
geom_dim = t.tensor([[x_min, x_max],[y_min, y_max],[z_min, z_max]])

In [20]:
dx = I.shape[0]/zooms[0]
dy = I.shape[1]/zooms[1]
dz = I.shape[2]/zooms[2]

tensor(592.4572)
tensor(592.4572)
tensor(1.1364)


In [13]:
print(I.shape[0]/zooms[0])

tensor(592.4572)


In [27]:
volume = t.zeros(576,576,5)

In [58]:
a = t.linspace(-5,5,2)
b = t.linspace(0,10,1)
c = t.linspace(1,2,3)
a_g, b_g, c_g = t.meshgrid(a,b,c)
t.stack((t.flatten(a_g),t.flatten(b_g), t.flatten(c_g)),dim = 0)

tensor([[-5.0000, -5.0000, -5.0000,  5.0000,  5.0000,  5.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.0000,  1.5000,  2.0000,  1.0000,  1.5000,  2.0000]])

In [68]:
def create_voxel_mesh(geometry,n_voxels):
    """Function to create the voxel mesh. i.e. the target volume X
    Inputs: 
    geometry: [[x_min, x_max],[y_min, y_max],[z_min, z_max]]
    voxels[n_x, n_y, n_z] numer of voxels in each dimension
    """
    #lenths
    [l_x,l_y,l_z] = geometry[:,0] - geometry[:,1]
    x_lin, y_lin, z_lin = t.linspace(geometry[0,0],geometry[0,1],n_voxels[0]),  t.linspace(geometry[1,0],geometry[1,1], n_voxels[1]), t.linspace(geometry[2,0],geometry[2,1], n_voxels[2])
    x_grid, y_grid, z_grid = t.meshgrid(x_lin, y_lin, z_lin)
    #coordinates
    coordinates = t.stack((t.flatten(x_grid),t.flatten(y_grid),t.flatten(z_grid)),dim = 0)
    #1 for "voxel structure" 0 for initial intensity value
    add_on = t.tensor([[1],[0]]).repeat(1,coordinates.shape[1])
    voxels = t.cat((coordinates,add_on),0)
    return voxels
    

In [96]:
geometry = t.tensor([[0.5,10],[-0.1,10],[-5,5]])
n_voxels = [2,1,3]

In [97]:
create_voxel_mesh(geometry,n_voxels)

tensor([[ 0.5000,  0.5000,  0.5000, 10.0000, 10.0000, 10.0000],
        [-0.1000, -0.1000, -0.1000, -0.1000, -0.1000, -0.1000],
        [-5.0000,  0.0000,  5.0000, -5.0000,  0.0000,  5.0000],
        [ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000]])

In [363]:
I = t.rand(2,3,5)
first_stack = stack.stack(I,t_affine,1)
F = t.eye(4).unsqueeze(2).repeat(1,1,5)
F.shape


torch.Size([4, 4, 5])

In [364]:
x_lin,y_lin,k_lin = t.linspace(0,I.shape[0]-1,I.shape[0]), t.linspace(0,I.shape[1]-1,I.shape[1]), t.linspace(0,I.shape[2]-1,I.shape[2])
x_grid, y_grid = t.meshgrid(x_lin, y_lin)
coordinates = t.stack((t.flatten(x_grid), t.flatten(y_grid)), dim = 0)
add_on = t.tensor([[0],[1]]).repeat(1,coordinates.shape[1])
pixels = t.cat((coordinates, add_on),0)
output = t.einsum('ijk,jl->ilk',F,pixels)
#switch order for consistency with 2d coordinates
k_grid, x_grid, y_grid = t.meshgrid(k_lin, x_lin, y_lin)
coordinates_3d = t.stack((t.flatten(x_grid), t.flatten(y_grid), t.flatten(k_grid)), dim = 0).int().numpy()
values = I[coordinates_3d][None,:]
output = output.reshape(output.shape[0],output.shape[1]*output.shape[2])
output = t.cat((output,values),0)

In [357]:
output = t.einsum('ijk,jl->ilk',F,pixels)

#switch order for consistency with 2d coordinates
k_grid, x_grid, y_grid = t.meshgrid(k_lin, x_lin, y_lin)
coordinates_3d = t.stack((t.flatten(x_grid), t.flatten(y_grid), t.flatten(k_grid)), dim = 0).int().numpy()
values = I[coordinates_3d][None,:]
print(values.shape)

output = output.reshape(output.shape[0],output.shape[1]*output.shape[2])
print(output.shape)
output = t.cat((output,values),0)
print(output)

torch.Size([1, 30])
torch.Size([4, 30])
tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 2.0000, 2.0000, 2.0000, 2.0000, 2.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 2.0000, 2.0000,
         2.0000, 2.0000, 2.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000],
        [1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000,

In [365]:
print(coordinates_3d)
print(coordinates)

[[0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1 0 0 0 1 1 1]
 [0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2 0 1 2]
 [0 0 0 0 0 0 1 1 1 1 1 1 2 2 2 2 2 2 3 3 3 3 3 3 4 4 4 4 4 4]]
tensor([[0., 0., 0., 1., 1., 1.],
        [0., 1., 2., 0., 1., 2.]])


In [366]:
I

tensor([[[0.0556, 0.8104, 0.8539, 0.6413, 0.3888],
         [0.4669, 0.0687, 0.8441, 0.8214, 0.1583],
         [0.6239, 0.8173, 0.5300, 0.1865, 0.1495]],

        [[0.8402, 0.5287, 0.7594, 0.7014, 0.9397],
         [0.5108, 0.3857, 0.8449, 0.0427, 0.8482],
         [0.3912, 0.7829, 0.3642, 0.2466, 0.5527]]])

In [367]:
output.shape

torch.Size([5, 30])

In [368]:
output[4,:]

tensor([0.0556, 0.4669, 0.6239, 0.8402, 0.5108, 0.3912, 0.8104, 0.0687, 0.8173,
        0.5287, 0.3857, 0.7829, 0.8539, 0.8441, 0.5300, 0.7594, 0.8449, 0.3642,
        0.6413, 0.8214, 0.1865, 0.7014, 0.0427, 0.2466, 0.3888, 0.1583, 0.1495,
        0.9397, 0.8482, 0.5527])

In [369]:
I[1,0,4]

tensor(0.9397)

In [370]:
I[1,1,0]

tensor(0.5108)